Ref: https://github.com/microsoft/semantic-kernel/tree/main/dotnet/notebooks

In [2]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Microsoft.Extensions.Logging.Console"

Installed Packages Microsoft.Extensions.Logging, 9.0.10 Microsoft.Extensions.Logging.Console, 9.0.10 Microsoft.SemanticKernel, 1.66.0

## 1) Import packages

In [11]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

using Microsoft.Extensions.Logging;
using Microsoft.Extensions.DependencyInjection;

## 2) Add AI services
- In this example, we used Azure OpenAI, but you can use any other chat completion service.

In [ ]:
// Create kernel
var builder = Kernel.CreateBuilder();
string modelId = "";
string endpoint = "";
string apiKey = "";
builder.AddAzureOpenAIChatCompletion(modelId, endpoint, apiKey);

## 3) Add enterprise services
- One of the main benefits of using Semantic Kernel is that it supports enterprise-grade services. In this sample, we added the logging service to the kernel to help debug the AI agent.

In [17]:
builder.Services.AddLogging(services => services.AddConsole().SetMinimumLevel(LogLevel.Trace));

## 4) Build the kernel and retrieve services
- Once the services have been added, we then build the kernel and retrieve the chat completion service for later use.

In [18]:
Kernel kernel = builder.Build();

// Retrieve the chat completion service
var chatCompletionService = kernel.Services.GetRequiredService<IChatCompletionService>();

## 5) Add plugins
- With plugins, can give your AI agent the ability to run your code to retrieve information from external sources or to perform actions. In the above example, we added a plugin that allows the AI agent to interact with a light bulb. Below, we'll show you how to create this plugin.

### a) Create a native plugin
Below, you can see that creating a native plugin is as simple as creating a new class.

- In this example, we've created a plugin that can manipulate a light bulb. While this is a simple example, this plugin quickly demonstrates how you can support both...
1. Retrieval Augmented Generation (RAG) by providing the AI agent with the state of the light bulb
2. And task automation by allowing the AI agent to turn the light bulb on or off.
- In your own code, you can create a plugin that interacts with any external service or API to achieve similar results.

In [19]:
using System.ComponentModel;
using System.Text.Json.Serialization;
using Microsoft.SemanticKernel;

public class LightsPlugin
{
   // Mock data for the lights
   private readonly List<LightModel> lights = new()
   {
      new LightModel { Id = 1, Name = "Table Lamp", IsOn = false },
      new LightModel { Id = 2, Name = "Porch light", IsOn = false },
      new LightModel { Id = 3, Name = "Chandelier", IsOn = true }
   };

   [KernelFunction("get_lights")]
   [Description("Gets a list of lights and their current state")]
   public async Task<List<LightModel>> GetLightsAsync()
   {
      return lights;
   }

   [KernelFunction("change_state")]
   [Description("Changes the state of the light")]
   public async Task<LightModel?> ChangeStateAsync(int id, bool isOn)
   {
      var light = lights.FirstOrDefault(light => light.Id == id);

      if (light == null)
      {
         return null;
      }

      // Update the light with the new state
      light.IsOn = isOn;

      return light;
   }
}

public class LightModel
{
   [JsonPropertyName("id")]
   public int Id { get; set; }

   [JsonPropertyName("name")]
   public string Name { get; set; }

   [JsonPropertyName("is_on")]
   public bool? IsOn { get; set; }
}

### b) Add the plugin to the kernel

In [20]:
// Add the plugin to the kernel
kernel.Plugins.AddFromType<LightsPlugin>("Lights");

## 6) Planning

Semantic Kernel leverages function calling–a native feature of most LLMs–to provide planning. With function calling, LLMs can request (or call) a particular function to satisfy a user's request. Semantic Kernel then marshals the request to the appropriate function in your codebase and returns the results back to the LLM so the AI agent can generate a final response.

To enable automatic function calling, we first need to create the appropriate execution settings so that Semantic Kernel knows to automatically invoke the functions in the kernel when the AI agent requests them.

In [21]:
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

## 10) Invoke

Finally, we invoke the AI agent with the plugin. The sample code demonstrates how to generate a non-streaming response, but you can also generate a streaming response by using the GetStreamingChatMessageContentAsync method.

In [30]:
// Create chat history
var history = new ChatHistory();

var userInput = "Turn on the porch light and turn off the chandelier.";

// Add user input
history.AddUserMessage(userInput);

// Get the response from the AI
var result = await chatCompletionService.GetChatMessageContentAsync(
    history,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel
);

In [31]:
result

{\n "type": "function",\n "function": [\n {"name": "Lights-change_state", "arguments": {"id": 1, "isOn": true}},\n {"name": "Lights-change_state", "arguments": {"id": 2, "isOn": false}}\n ]\n} ToolCalls (empty) AuthorName <null> Role Assistant Label Assistant Content {
 "type": "function",
 "function": [
 {"name": "Lights-change_state", "arguments": {"id": 1, "isOn": true}},
 {"name": "Lights-change_state", "arguments": {"id": 2, "isOn": false}}
 ]
} Items [ {
 "type": "function",
 "function": [
 {"name": "Lights-change_state", "arguments": {"id": 1, "isOn": true}},
 {"name": "Lights-change_state", "arguments": {"id": 2, "isOn": false}}
 ]
} ] Count 1 (values) index value 0 {\n "type": "function",\n "function": [\n {"name": "Lights-change_state", "arguments": {"id": 1, "isOn": true}},\n {"name": "Lights-change_state", "arguments": {"id": 2, "isOn": false}}\n ]\n} Text {
 "type": "function",
 "function": [
 {"name": "Lights-change_state", "arguments": {"id": 1, "isOn": true}},
 {"name": "Lights-change_state", "arguments": {"id": 2, "isOn": false}}
 ]
} Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 MimeType <null> InnerContent <null> ModelId <null> Metadata <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 Source <null> MimeType <null> InnerContent OpenAI.Chat.ChatCompletion CreatedAt 2025-10-21 09:52:37Z FinishReason Stop ContentTokenLogProbabilities [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) RefusalTokenLogProbabilities [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) Role Assistant Content index value 0 OpenAI.Chat.ChatMessageContentPart Kind Text Text {
 "type": "function",
 "function": [
 {"name": "Lights-change_state", "arguments": {"id": 1, "isOn": true}},
 {"name": "Lights-change_state", "arguments": {"id": 2, "isOn": false}}
 ]
} ImageUri <null> ImageBytes <null> ImageBytesMediaType <null> InputAudioBytes <null> InputAudioFormat <null> FileId <null> FileBytes <null> FileBytesMediaType <null> Filename <null> ImageDetailLevel <null> Refusal <null> ToolCalls (empty) Refusal <null> FunctionCall <null> OutputAudio <null> Annotations (empty) Id chatcmpl-4f276713-9442-4ca6-969b-aad75c57e710 Model phi4-mini-instruct ServiceTier <null> SystemFingerprint <null> Usage OpenAI.Chat.ChatTokenUsage OutputTokenCount 65 InputTokenCount 499 TotalTokenCount 564 OutputTokenDetails <null> InputTokenDetails <null> ModelId Phi-4-mini-instruct Metadata key type value Id System.String chatcmpl-4f276713-9442-4ca6-969b-aad75c57e710 CreatedAt System.DateTimeOffset 2025-10-21 09:52:37Z SystemFingerprint <null> <null> Usage OpenAI.Chat.ChatTokenUsage OpenAI.Chat.ChatTokenUsage OutputTokenCount 65 InputTokenCount 499 TotalTokenCount 564 OutputTokenDetails <null> InputTokenDetails <null> Refusal <null> <null> FinishReason System.String Stop ContentTokenLogProbabilities OpenAI.ChangeTrackingList<OpenAI.Chat.ChatTokenLogProbabilityDetails>